In [58]:
import pandas as pd
import os

In [59]:
files_dir = "world_bank_projects"
files_list = os.listdir(files_dir)

In [60]:
tables = []
for f in files_list:
    if f[-4:] == 'xlsx':
        table = pd.read_excel(os.path.join(files_dir,f),skiprows=1)
        tables.append(table)

In [61]:
tables[0].head()

,Project Id,Region,Country,Project Status,Project Name,Project Development Objective,Implementing Agency,Consultant Services Required,Project URL,Board Approval Date,...,Grant Amount,Borrower,Lending Instrument,Environmental Assessment Category,Environmental and Social Risk,Sector 1,Sector 2,Sector 3,Theme 1,Theme 2
0,P004092,East Asia and Pacific,Republic of Korea,Closed,Gojeong Power Project,NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,NaN,...,0,NaN,Specific Investment Loan,NaN,NaN,(Historic)Thermal,NaN,NaN,NaN,NaN
1,P004233,East Asia and Pacific,Malaysia,Closed,Population Project,NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,NaN,...,0,NaN,Sector Investment and Maintenance Loan,NaN,NaN,"(Historic)Primary health, including reproducti...",NaN,NaN,NaN,NaN
2,P009174,Europe and Central Asia,"Macedonia, Republic of North",Closed,Agriculture & Agroindustry 2 Project (Macedonia),NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,1980-02-01T00:00:00Z,...,0,NaN,Specific Investment Loan,C,NaN,(Historic)Agricultural credit,NaN,NaN,NaN,NaN
3,P007361,Latin America and Caribbean,Republic of Honduras,Closed,Industrial Credit Project (02),NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,1981-12-22T00:00:00Z,...,0,NaN,Financial Intermediary Loan,NaN,NaN,(Historic)Financial sector development,NaN,NaN,NaN,NaN
4,P003277,Africa East,Republic of Zimbabwe,Closed,Railway Development Project,NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,1983-08-02T00:00:00Z,...,0,NaN,Technical Assistance Loan,NaN,NaN,Railways,NaN,NaN,NaN,NaN


In [62]:
table = pd.concat(tables,ignore_index=True)

In [64]:
table['IBRD Commitment'] = table['IBRD Commitment'].apply(lambda x: x.replace(',',''))
table['IBRD Commitment'] = table['IBRD Commitment'].astype(float)

In [65]:
table['IDA Commitment'] = table['IDA Commitment'].apply(lambda x: x.replace(',',''))
table['IDA Commitment'] = table['IDA Commitment'].astype(float)

In [66]:
table['Total IDA and IBRD Commitment'] = table['Total IDA and IBRD Commitment'].apply(lambda x: x.replace(',',''))
table['Total IDA and IBRD Commitment'] = table['Total IDA and IBRD Commitment'].astype(float)

In [57]:
table.to_csv(os.path.join(files_dir,"world_bank_projects.csv"),index=False)


It turns out that the Excel files are for some reason not formed correctly by the World Bank site. Instead, we use the JSON-based API: 
https://search.worldbank.org/api/v2/projects
  ?format=json
  &fl=id,regionname,countryname,projectstatusdisplay,project_name,countryshortname,pdo,
  impagency,cons_serv_reqd_ind,url,boardapprovaldate,closingdate,projectfinancialtype,
  curr_project_cost,ibrdcommamt,idacommamt,totalamt,grantamt,borrower,lendinginstr,
  envassesmentcategorycode,esrc_ovrl_risk_rate,sector1,sector2,sector3,
  theme1,theme2,%20%20status,totalcommamt,proj_last_upd_date
  &apilang=en
  &projectfinancialtype_exact=IBRD
  &rows=4000&os=0

In [113]:
wb_json = pd.read_json(os.path.join(files_dir,'wbapiall_clean.json'),orient='index')

In [114]:
wb_json.describe()

,id,regionname,countryname,lendinginstr,projectstatusdisplay,status,project_name,boardapprovaldate,ibrdcommamt,idacommamt,...,theme1,url,totalcommamt,cons_serv_reqd_ind,closingdate,envassesmentcategorycode,sector2,sector3,project_abstract,theme2
count,3697,3697,3697,3678,3697,3697,3697,3685,3697,3697,...,3697,3697,3697,190,3424,2976,2943,1994,3252,2783
unique,3697,8,106,16,2,2,3526,1872,966,129,...,886,3697,976,3,640,7,1276,875,3194,912
top,P166279,Latin America and Caribbean,[People's Republic of China],Specific Investment Loan,Closed,Closed,Urban Transport Project,2001-06-26T00:00:00Z,"100,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"100,000,000",Y,2021-12-31T00:00:00Z,B,{'Name': 'Central Government (Central Agencies...,"{'Name': 'Sub-National Government', 'Percent': 5}",{'cdata': 'This report combines five emergency...,{'Name': 'Infrastructure services for private ...
freq,1,1248,316,1475,2953,2953,6,10,242,3461,...,496,1,242,104,74,1373,31,22,5,32


In [115]:
wb_json.head()

,id,regionname,countryname,lendinginstr,projectstatusdisplay,status,project_name,boardapprovaldate,ibrdcommamt,idacommamt,...,theme1,url,totalcommamt,cons_serv_reqd_ind,closingdate,envassesmentcategorycode,sector2,sector3,project_abstract,theme2
P172024,P172024,Europe and Central Asia,[Republic of Croatia],Investment Project Financing,Active,Active,Helping Enterprises Access Liquidity in the Re...,2021-05-14T00:00:00Z,"242,100,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"242,100,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
P173975,P173975,Europe and Central Asia,[Georgia],Investment Project Financing,Active,Active,"Georgia Relief and Recovery for Micro, Small, ...",2021-05-11T00:00:00Z,"102,900,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"102,900,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
P175895,P175895,Europe and Central Asia,[Ukraine],Investment Project Financing,Active,Active,Ukraine Emergency COVID-19 Response and Vaccin...,2021-05-10T00:00:00Z,"90,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"90,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
P171050,P171050,Europe and Central Asia,[Ukraine],Investment Project Financing,Active,Active,Ukraine Improving Higher Education for Results...,2021-05-05T00:00:00Z,"200,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"200,000,000",N,NaN,NaN,NaN,NaN,NaN,NaN
P175138,P175138,Latin America and Caribbean,[Argentine Republic],Investment Project Financing,Active,Active,Buenos Aires &#8211; Mitre Passenger Railway L...,2021-04-30T00:00:00Z,"347,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"347,000,000",Y,2026-06-30T00:00:00Z,NaN,NaN,NaN,NaN,NaN


In [116]:
wb_json.ibrdcommamt = wb_json.ibrdcommamt.apply(lambda x: x.replace(',',''))
wb_json.ibrdcommamt = wb_json.ibrdcommamt.astype(float)

In [117]:
import numpy as np

In [118]:
wb_json['sector1_name'] = [x['Name'] for x in wb_json['sector1']]
wb_json['sector1_pct'] = [x['Percent'] for x in wb_json['sector1']]
wb_json['sector2_name'] = [x['Name'] if type(x) == dict else np.nan for x in wb_json['sector2']]
wb_json['sector2_pct'] = [x['Percent'] if type(x) == dict else np.nan for x in wb_json['sector2']]
wb_json['sector3_name'] = [x['Name'] if type(x) == dict else np.nan for x in wb_json['sector3']]
wb_json['sector3_pct'] = [x['Percent'] if type(x) == dict else np.nan for x in wb_json['sector3']]

In [119]:
import pickle

In [120]:
wb_json.boardapprovaldate = wb_json.boardapprovaldate.apply(pd.Timestamp)

In [121]:
wb_json.boardapprovaldate.describe()

<ipython-input-121-c98746017877>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  wb_json.boardapprovaldate.describe()


count                          3685
unique                         1872
top       2001-06-26 00:00:00+00:00
freq                             10
first     1980-02-01 00:00:00+00:00
last      2021-05-14 00:00:00+00:00
Name: boardapprovaldate, dtype: object

In [122]:
wb_json.to_pickle(os.path.join(files_dir,"world_bank_projects_json.pkl"))